In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
page = requests.get("https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldYear&year=2021")
soup = BeautifulSoup(page.content, 'html.parser')

In [13]:
t1 = soup.find(class_="t-chart")

In [22]:
t2 = t1.find_all(class_="oddrow")

In [24]:
t3 = t1.select(".oddrow .text_view_data")

In [26]:
valueList = t3.get_text()

AttributeError: 'list' object has no attribute 'get_text'